In [1]:
import numpy as np
import pandas as pd
import os

%matplotlib inline
import matplotlib.pyplot as plt

import LiwcTrie as lt
from Liwc import LiwcAnalyzer

from fbdp import FBDesktopParser

[nltk_data] Downloading package wordnet to /home/Phillip/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/Phillip/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/Phillip/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
#Load facebook desktop parser
fbdp = FBDesktopParser()
fbdp.parse_posts()
fbdp.extract_features(bag_of_words=False, lemmatize=True)

,article_host,article_name,article_subtitle,hashtags,img-label,img_src,linked_profiles,links,text,timestamp,...,num_periods,percent_periods,num_exclamations,percent_exclamations,num_questionms,percent_questionms,num_equals,percent_equals,num_dollars,percent_dollars
0,stopmandatoryvaccination.com,Get These Vaccine Books & DVDs Before Amazon B...,"This is where you start, right here, with thes...",[],Amazon Censorship Of Vaccine Books,./Stop Mandatory Vaccination - Posts_files/537...,[],[],"Get educated. Seriously, read several books an...","3/3/19, 4:27 PM",...,5,0.042017,0,0.000000,0,0.000000,0,0.0,0,0.0
1,None,None,None,[],None,./Stop Mandatory Vaccination - Posts_files/530...,[],[http://bit.ly/Thyroid_Reset],"Your host of The Thyroid Reset Summit, Dr. Jus...","3/5/19, 1:00 PM",...,2,0.027027,0,0.000000,0,0.000000,0,0.0,0,0.0
2,None,None,None,[],None,None,[],[],If the US Federal Govt mandates vaccines will ...,"3/5/19, 12:28 PM",...,0,0.000000,0,0.000000,1,0.071429,0,0.0,0,0.0
3,physiciansforinformedconsent.org,physiciansforinformedconsent.org,None,[],None,None,[Health Choice Minnesota],[https://physiciansforinformedconsent.org/…/PI...,🚨 Urgent 🚨 U.S. Senate public hearing on “Vac...,"3/5/19, 12:26 PM",...,18,0.019397,0,0.000000,2,0.002155,0,0.0,0,0.0
4,None,None,None,"[HumanRIGHTS, paid, demonstration, corruption,...",None,./Stop Mandatory Vaccination - Posts_files/536...,"[Stef Miharas, Laira De La Vega]",[],"Here is what corruption looks like, up close a...","3/5/19, 12:21 PM",...,12,0.036923,0,0.000000,2,0.006154,0,0.0,0,0.0
5,None,None,None,[],None,None,[Children's Health Defense],[],Vaccines cause seizures. But don't concern you...,"3/5/19, 12:18 PM",...,3,0.055556,0,0.000000,0,0.000000,0,0.0,0,0.0
6,stopmandatoryvaccination.com,Your Child’s Fever Is An Essential Immune Resp...,"Don't Panic, and Don't Lower it! Find out why!",[],"Dr. Judith Thompson, ND",./Stop Mandatory Vaccination - Posts_files/saf...,[],[],"If your child gets a fever, don't panic and do...","3/5/19, 12:00 PM",...,0,0.000000,1,0.062500,0,0.000000,0,0.0,0,0.0
7,None,None,None,[],None,./Stop Mandatory Vaccination - Posts_files/523...,[],[http://bit.ly/DandAeBook],The Harvard Health Blog reports that two omega...,"3/5/19, 10:00 AM",...,2,0.025641,0,0.000000,0,0.000000,0,0.0,0,0.0
8,childrenshealthdefense.org,CDC’s ACIP Considers Expanding Recommendations...,CDC’s Advisory Committee for Immunization Prac...,[],Human Papilloma Virus vaccine with syringe in ...,./Stop Mandatory Vaccination - Posts_files/saf...,[],[],The CDC Advisory Committee for Immunization Pr...,"3/5/19, 8:00 AM",...,2,0.043478,0,0.000000,0,0.000000,0,0.0,0,0.0
9,None,None,None,[],None,None,"[Zoe Harcombe, Aseem Malhotra]",[],Check it...Dear friends yesterday there was a ...,"3/5/19, 3:11 AM",...,12,0.042857,2,0.007143,0,0.000000,0,0.0,0,0.0


---------

bigrams and word pairings (using lemmatized text)

In [3]:
import re
from itertools import combinations, islice
from collections import Counter
from nltk.corpus import stopwords

In [4]:
stop_words = set(stopwords.words('english')) 
w = fbdp.posts.text_tokenized_lemmatized.apply(
    lambda words: [word for word in words if word not in stop_words and not word.isdigit()])

def count_bigrams(words):
    return Counter(zip(words, islice(words, 1, None)))

def pairs(words):
    return list(combinations(set(words), 2))

In [5]:
w.apply(count_bigrams).sum().most_common()

SystemError: <built-in function isinstance> returned a result with an error set

In [ ]:
counter = Counter()
for ind, val in w.apply(pairs).iteritems():
    counter.update(val)
counter.most_common()

-----
punctuation

In [ ]:
from string import punctuation

In [ ]:
def count_punctuation(text):
    counts = {}
    for c in list(punctuation):
        counts[c] = text.count(c)
    return counts

In [ ]:
counter = Counter()
for ind, val in fbdp.posts.text.apply(count_punctuation).iteritems():
    counter.update(val)

counter.most_common()

In [ ]:
# avg
[(c[0], c[1] / fbdp.posts.text.shape[0]) for c in counter.most_common()]

--------------
sentiment analysis

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
analyser = SentimentIntensityAnalyzer()

In [ ]:
sentiments = fbdp.posts.text.apply(analyser.polarity_scores)
sentiments

In [ ]:
counter = Counter()
for ind, val in sentiments.iteritems():
    counter.update(val)
                   
counter.most_common()

In [ ]:
p = 0 # positives
n = 0 # negatives
for s in sentiments:
    if s['compound'] > 0:
        p += 1
    elif s['compound'] < 0:
        n += 1
p, n